In [2]:
import numpy as np
import re
import random
import time
import os
import sys

from common_utils import strToSuffix



In [3]:
np.array([1,2,3], dtype='float32')

array([1., 2., 3.], dtype=float32)

In [2]:
import argparse
parser = argparse.ArgumentParser(description='Arguments for DANIEL_FJSP')

# args for env
parser.add_argument('--n_j', type=int, default=10, help='Number of jobs of the instance')
parser.add_argument('--n_m', type=int, default=5, help='Number of machines of the instance')
parser.add_argument('--n_op', type=int, default=60, help='Number of operations of the instance')
parser.add_argument('--low', type=int, default=1, help='Lower Bound of processing time(PT)')
parser.add_argument('--high', type=int, default=99, help='Upper Bound of processing time')
parser.add_argument('--data_suffix', type=str, default='mix', help='Suffix of the data')
parser.add_argument('--op_per_mch_min', type=int, default=1,
                    help='Minimum number of compatible machines for each operation')
parser.add_argument('--op_per_mch_max', type=int, default=5,
                    help='Maximum number of compatible machines for each operation')
parser.add_argument('--op_per_job', type=float, default=2,
                    help='Number of operations per job, default 0, means the number equals m')

config = parser.parse_args(args=[])



In [3]:


n_j=None
n_m=None

if n_j is None:n_j = config.n_j 
if n_m is None:n_m = config.n_m 
if config.op_per_job == 0:
    op_per_job = n_m
else:
    op_per_job = config.op_per_job

low = config.low
high = config.high
data_suffix = config.data_suffix

op_per_mch_min = 1
if data_suffix == "nf":
    op_per_mch_max = 1
elif data_suffix == "mix":
    op_per_mch_max = n_m
else:
    op_per_mch_min = config.op_per_mch_min
    op_per_mch_max = config.op_per_mch_max
if op_per_mch_min < 1 or op_per_mch_max > n_m:
    print(f'Error from Instance Generation: [{op_per_mch_min},{op_per_mch_max}] '
            f'with num_mch : {n_m}')
    sys.exit()


In [4]:

n_op = int(n_j * op_per_job)
job_length = np.full(shape=(n_j,), fill_value=op_per_job, dtype=int)
op_use_mch = np.random.randint(low=op_per_mch_min, high=op_per_mch_max + 1,
                                size=n_op)

op_per_mch = np.mean(op_use_mch)
op_pt = np.random.randint(low=low, high=high + 1, size=(n_op, n_m))
print(op_pt.shape)
for row in range(op_pt.shape[0]):
    mch_num = int(op_use_mch[row])
    if mch_num < n_m:
        inf_pos = np.random.choice(np.arange(0, n_m), n_m - mch_num, replace=False)
        op_pt[row][inf_pos] = 0

job_length, op_pt, len(op_pt), op_per_mch


(20, 5)


(array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 array([[14, 65, 35, 86, 80],
        [53, 88, 98,  0, 70],
        [66, 46,  0, 64,  0],
        [ 0, 66,  0,  0,  0],
        [ 0,  0,  4,  0,  0],
        [43, 92, 60,  6, 53],
        [82,  0, 11,  0, 57],
        [73,  0, 82, 40,  0],
        [16,  0,  0, 80, 52],
        [ 0, 60, 63, 64, 44],
        [30, 24,  0, 10,  0],
        [ 0, 43, 15,  0, 43],
        [ 0, 83,  0,  0, 53],
        [47,  6, 68, 31,  0],
        [ 0,  0,  0, 50,  0],
        [ 0,  0,  6, 27,  4],
        [ 1, 34, 69, 75,  3],
        [ 0, 89, 79, 75, 20],
        [31, 35,  0,  0, 18],
        [ 0, 95,  1,  0,  0]]),
 20,
 3.1)

In [5]:

# Modify here for variable number of operations per job
job_length = np.random.randint(low=1, high=op_per_job + 1, size=n_j)
n_op = np.sum(job_length)

op_use_mch = np.random.randint(low=op_per_mch_min, high=op_per_mch_max + 1, size=n_op)
op_per_mch = np.mean(op_use_mch)
op_pt = np.random.randint(low=low, high=high + 1, size=(n_op, n_m))

for row in range(op_pt.shape[0]):
    mch_num = int(op_use_mch[row])
    if mch_num < n_m:
        inf_pos = np.random.choice(np.arange(0, n_m), n_m - mch_num, replace=False)
        op_pt[row][inf_pos] = 0

job_length, op_pt, len(op_pt),  op_per_mch

(array([2, 2, 2, 2, 1, 2, 1, 2, 2, 2]),
 array([[65,  0, 42, 36,  0],
        [86,  0,  0,  0,  0],
        [32,  0,  0,  0,  0],
        [ 0, 89, 12, 67,  1],
        [80, 52, 70, 15, 22],
        [ 0, 57,  0, 38, 19],
        [ 0,  0, 95, 11,  0],
        [16,  2, 62, 70, 91],
        [44, 54, 87, 23,  0],
        [ 0, 14, 49, 52, 63],
        [71,  5, 13, 31,  2],
        [94,  0,  0,  0, 14],
        [ 3,  0,  0,  0, 51],
        [ 0, 80, 73, 59, 83],
        [ 9, 19,  0, 60, 92],
        [95, 43, 85,  0, 20],
        [ 0, 90,  9, 14, 84],
        [66, 46, 46, 84, 91]]),
 18,
 3.4444444444444446)

In [12]:
import numpy as np

def generate_flexible_job_shop(num_jobs, num_machines):
    job_length = np.random.randint(2, 6, size=num_jobs)
    total_operations = sum(job_length)
    op_pt = np.zeros((total_operations, num_machines))
    total_machines_per_op = 0

    op_index = 0
    for job in range(num_jobs):
        for _ in range(job_length[job]):
            num_machines_for_op = np.random.randint(1, num_machines + 1)
            total_machines_per_op += num_machines_for_op
            machines_for_op = np.random.choice(num_machines, num_machines_for_op, replace=False)
            
            for machine in machines_for_op:
                op_pt[op_index, machine] = np.random.randint(1, 99)  # Random processing time

            op_index += 1

    op_per_mch = total_machines_per_op / total_operations
    return job_length, op_pt, op_per_mch

# Example call
num_jobs = 10
num_machines = 5
job_length, op_pt, op_per_mch = generate_flexible_job_shop(num_jobs, num_machines)
job_length, op_pt, op_per_mch

(array([2, 2, 3, 5, 3, 5, 3, 5, 2, 5]),
 array([[49., 87., 28., 96.,  0.],
        [21., 11., 47.,  0., 54.],
        [61.,  0., 38.,  0.,  0.],
        [61.,  0.,  0.,  0., 15.],
        [ 0.,  0., 27., 22.,  0.],
        [87.,  0.,  0., 94.,  0.],
        [ 0., 55., 33., 60., 64.],
        [89., 50.,  0.,  0.,  0.],
        [ 0., 20.,  0., 21.,  0.],
        [ 0., 74., 56.,  0., 89.],
        [ 0., 56.,  0.,  0., 93.],
        [ 0., 47., 95.,  0.,  0.],
        [79.,  0.,  4.,  2., 58.],
        [68., 47., 88., 32.,  0.],
        [ 0., 69.,  0.,  6.,  0.],
        [49.,  0., 66.,  0.,  0.],
        [67., 92., 95.,  0., 79.],
        [87., 97., 17., 23., 18.],
        [ 0., 49., 75.,  0., 49.],
        [ 2.,  0., 64.,  0., 75.],
        [ 0.,  0.,  0., 64.,  0.],
        [92., 91.,  0., 52., 13.],
        [72., 82., 95., 63., 27.],
        [80.,  0., 53.,  0.,  0.],
        [93., 27., 77., 98., 55.],
        [ 0.,  0.,  0.,  0., 86.],
        [30., 47., 69., 56., 87.],
        [ 0., 3

In [ ]:
job_init = 10   #工件数量
num_mas = 5     #机器数
opes_per_job_min, opes_per_job_max = 5, 10  
mas_per_ope_min, mas_per_ope_max = 1, 5    #每个机器最大的加工数量
num_jobs = job_init

nums_ope = [random.randint(opes_per_job_min, opes_per_job_max) for _ in range(num_jobs)]  ##随机生成每个工件的操作数
num_opes = sum(nums_ope)
nums_option = [random.randint(mas_per_ope_min, mas_per_ope_max) for _ in range(num_opes)] ##随机生成每个操作可行的机器数量
num_options = sum(nums_option)

ope_ma = []
for val in nums_option:
    ope_ma = ope_ma + sorted(random.sample(range(1, num_mas + 1), val))
proc_time = []

self.proc_times_mean = [random.randint(self.proctime_per_ope_min, self.proctime_per_ope_max) for _ in
                        range(self.num_opes)]
for i in range(len(self.nums_option)):
    low_bound = max(self.proctime_per_ope_min, round(self.proc_times_mean[i] * (1 - self.proctime_dev)))
    high_bound = min(self.proctime_per_ope_max, round(self.proc_times_mean[i] * (1 + self.proctime_dev)))
    proc_time_ope = [random.randint(low_bound, high_bound) for _ in range(self.nums_option[i])]
    self.proc_time = self.proc_time + proc_time_ope

self.num_ope_biass = [sum(self.nums_ope[0:i]) for i in range(self.num_jobs)]
self.num_ma_biass = [sum(self.nums_option[0:i]) for i in range(self.num_opes)]
line0 = '{0}\t{1}\t{2}\n'.format(self.num_jobs, self.num_mas, self.num_options / self.num_opes)
lines_doc = []
lines_doc.append('{0}\t{1}\t{2}'.format(self.num_jobs, self.num_mas, self.num_options / self.num_opes))
for i in range(self.num_jobs):
    flag = 0
    flag_time = 0
    flag_new_ope = 1
    idx_ope = -1
    idx_ma = 0
    line = []
    option_max = sum(self.nums_option[self.num_ope_biass[i]:(self.num_ope_biass[i] + self.nums_ope[i])])
    idx_option = 0
    while True:
        if flag == 0:
            line.append(self.nums_ope[i])
            flag += 1
        elif flag == flag_new_ope:
            idx_ope += 1
            idx_ma = 0
            flag_new_ope += self.nums_option[self.num_ope_biass[i] + idx_ope] * 2 + 1
            line.append(self.nums_option[self.num_ope_biass[i] + idx_ope])
            flag += 1
        elif flag_time == 0:
            line.append(self.ope_ma[self.num_ma_biass[self.num_ope_biass[i] + idx_ope] + idx_ma])
            flag += 1
            flag_time = 1
        else:
            line.append(self.proc_time[self.num_ma_biass[self.num_ope_biass[i] + idx_ope] + idx_ma])
            flag += 1
            flag_time = 0
            idx_option += 1
            idx_ma += 1
        if idx_option == option_max:
            str_line = " ".join([str(val) for val in line])
            lines_doc.append(str_line)
            break
job_length, op_pt = text_to_matrix(lines_doc)

job_length, op_pt


In [3]:

range_pose=0.3
target_pose=(2*np.random.rand(2)-1)*range_pose + [0.5, 0.5] # around center (0.5,0.5), range 0.3
screen_size=1000
target_pose=target_pose*screen_size
target_pose

# env=Reacher(target_pos=target_pose, render=True)
# return env, target_pose


array([257.77272383, 767.46712808])

In [4]:
np.random.rand(2)

array([0.99163272, 0.53982156])